In [69]:
# load modules
import numpy as np
import RNA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import random
import math
import time
import re
from collections import Counter,defaultdict
from Bio import SeqIO
from ast import literal_eval
from tqdm import tqdm

In [70]:
# define all functions

## basic stats
def seq_composition(s):
    L = len(s)
    nts_counts = Counter(s)
    A = nts_counts['A']
    C = nts_counts['C']
    G = nts_counts['G']
    U = nts_counts['U']
    GC = (C+G)/L
    return [A/L,C/L,G/L,U/L,
            A/C,A/G,A/U, C/G,C/U, G/U,
            GC]

## homopolymers
def find_homopolymer(seq):
    L = len(seq)
    homopolymers = [0,0,0,0]
    i = 0
    shortest_homopolymer_len = 4
    
    while i < L - shortest_homopolymer_len + 1:
        base = seq[i]
        homopolymer = base
        for j in range(i+1,L):
            if seq[j] == base:
                homopolymer += base
            else:
                break
        i = j
        if len(homopolymer) >= shortest_homopolymer_len:
            NT = list(set(homopolymer))[0]
            if NT == 'A': homopolymers[0] += 1
            elif NT == 'C': homopolymers[1] += 1
            elif NT == 'G': homopolymers[2] += 1
            elif NT == 'U': homopolymers[3] += 1
    
    return homopolymers

## entropy + parity
def shannon_entropy(kmer_counts):
    kmer_counts = [x for x in kmer_counts if x != 0]
    
    total_kmers = sum(kmer_counts)
    entropy = 0.0
    
    for count in kmer_counts:
        probability = count / total_kmers
        entropy -= probability * math.log2(probability)
    
    return entropy

def revcomp(seq):
    revcomp = ''
    for i in seq[::-1]:
        revcomp += {'A':'U','C':'G','G':'C','U':'A'}.get(i,i)
    return revcomp

def kmer_parity(seq):
    L = len(seq)
    results = []
    results_entropy = []
    for klen in range(1,10,2):
        kmer_counts = defaultdict(list)
        individual_kmer_counts = []
        for i in range(0,len(seq)+1-klen):
            kmer = seq[i:i+klen]
            rc_kmer = revcomp(kmer)
            canonical_kmer = min(kmer,rc_kmer)
            if canonical_kmer not in kmer_counts:
                kmer_counts[canonical_kmer] = [0,0]
            if kmer == canonical_kmer:
                kmer_counts[canonical_kmer][0] += 1
            else:
                kmer_counts[canonical_kmer][1] += 1
        dna_diff = 0
        for k,v in kmer_counts.items():
            dna_diff += abs(v[0]-v[1])
            individual_kmer_counts += v
        results.append(1 - dna_diff/L)
        results_entropy.append(shannon_entropy(individual_kmer_counts))
    return results + results_entropy

## primers
universal_primers = ['AGAGUUUGAUCCUGGCUCAG','AGAGUUUGAUC[AC]UGGCUCAG','ACUGCUGC[GC][CU]CCCGUAGGAGUCU','GACUCCUACGGGAGGC[AU]GCAG','GUAUUACCGCGGCUGCUGG','GUGCCAGC[AC]GCCGCGGUAA','GGAUUAGAUACCCUGGUA','GGACUAC[ACG][GC]GGGUAUCUAAU','CCGUCAAUUCCUUU[AG]AGUUU','UAAAACU[CU]AAA[GU]GAAUUGACGGG','[CU]AACGAGCGCAACCC','GGGUUGCGCUCGUUG','GGUUACCUUGUUACGACUU','CGGUUACCUUGUUACGACUU']
up_regexes = [re.compile(x) for x in universal_primers]

def find_universal_primers(s):
    matches = set()
    for regex in up_regexes:
        for match in regex.finditer(s):
            matches.add(match[0])
    return len(matches)

## frequent kmers
def proportion_kmers_present(seq, biomarker_kmers):
    present = 0
    for k in biomarker_kmers:
        if k in seq:
            present += 1
    return present/len(biomarker_kmers)

def find_biomarker_kmers(f, klen, min_freq_SILVA):
    kmer_counts = defaultdict(int)
    kmers_of_interest = set()
    total_seqs = 0
    
    for h,i in tqdm(enumerate(SeqIO.parse(f,'fasta'))):
        s = str(i.seq).upper().replace('T','U')
        total_seqs += 1
        kmers = set()
        for i in range(0,len(s)+1-klen):
            kmer = s[i:i+klen]
            kmers.add(kmer)
        for k in kmers:
            kmer_counts[k] += 1
    
    for k,v in sorted(kmer_counts.items(),key=lambda x:x[1]):
        if v/total_seqs > min_freq_SILVA:
            kmers_of_interest.add(k)
            
    return kmers_of_interest

## gapped k-mers
def load_gapped_kmers(k=11):
    # fetch kmers
    fn = './results/k-'+str(k)+'-300000.csv' # TODO: allow for different total_seqs
    kmers = dict()
    with open(fn, 'r') as f:
        for i, line in enumerate(f):
            kmers[i] = literal_eval(line.split(';')[0])
    return kmers

def make_kmer_regexes(kmers):
    k = len(kmers[0])
    kmer_regexes = []
    for km in kmers:
        re_str = ''
        chars = [x[0] for x in km]
        offsets = [x[1] for x in km]
        rel_offs = [offsets[i] - offsets[i-1] for i in range(1,k)]
        for i in range(k):
            re_str += chars[i]
            if i < k-1 and rel_offs[i] > 1:
                re_str += '.{' + str(rel_offs[i]-1) + '}'
        kmer_regexes.append(re.compile(re_str))
    return kmer_regexes

## secondary structure
def get_helices(base_pairs):
    if not base_pairs:
        return []

    helices = []
    current_helix = [base_pairs[0]]

    for i in range(1, len(base_pairs)):
        prev = base_pairs[i - 1]
        curr = base_pairs[i]

        # Check if it's part of the same helix (i.e., consecutive base pairs)
        if curr[0] == prev[0] + 1 and curr[1] == prev[1] - 1:
            current_helix.append(curr)
        else:
            helices.append(current_helix)
            current_helix = [curr]

    helices.append(current_helix)
    return helices

def get_base_pairs(dot_bracket):
    stack = []
    pairs = []
    for i, char in enumerate(dot_bracket):
        if char == '(':
            stack.append(i)
        elif char == ')':
            j = stack.pop()
            pairs.append((j, i))
    return sorted(pairs)

def predict_mfe_helices(rna_sequence):
    helix_length_threshold = 10
    fc = RNA.fold_compound(rna_sequence)
    structure, mfe = fc.mfe()
    base_pairs = get_base_pairs(structure)
    helices = get_helices(base_pairs)
    
    reported_helices = [len(helix) for helix in helices if len(helix) >= helix_length_threshold]
    
    return mfe, len(reported_helices)

## fingerprint?
nt_sequence = 'UACAUAAAAUAAUAGGCAGGAUAAUAGAGGCAGCGCGAACUGUGAAAAGGCAAUAGACUAAUAAGAUACAAAAUAAAUGCGGCGUAAUCCGUGCAGGUAUUGACUGAAGGAUGUAGGCUGACCCCGUAAAGUCCGGCUGG'
loci_sequence = [13,51,54,55,56,109,151,160,243,244,246,282,323,344,346,347,355,356,357,362,364,368,389,397,405,499,505,509,515,517,519,520,521,522,527,528,530,532,533,536,565,566,571,581,676,695,704,715,725,727,732,781,787,788,790,791,792,795,801,802,815,820,864,865,885,889,891,892,899,900,901,908,909,911,914,915,919,920,922,924,925,926,936,944,956,958,959,960,972,984,1050,1052,1053,1054,1055,1057,1058,1073,1093,1095,1199,1221,1227,1237,1315,1316,1318,1319,1337,1338,1339,1341,1347,1348,1349,1373,1379,1382,1391,1392,1394,1395,1397,1399,1403,1405,1406,1418,1492,1493,1494,1495,1496,1501,1504,1505,1509,1512,1517,1526]
def universally_conserved_nucleotide_fingerprint():
    ref_seq_N = ''
    
    for i in range(1, 1501):
        if i not in loci_sequence:
            ref_seq_N += 'N'
        else:
            ref_seq_N += nt_sequence[loci_sequence.index(i)]
    
    clustered_nt_sequence = []
    
    for i in ref_seq_N.split('N'):
        if i != '': clustered_nt_sequence.append(i)
    
    return clustered_nt_sequence

def find_conserved_nucleotide_fingerprint(query_sequence, nt_sequence):
    found_univ_conserved_nts = []
    start = 0

    for i in nt_sequence:
        loci = query_sequence.find(i,start,len(query_sequence)+1)
        if loci != -1:
            found_univ_conserved_nts.append(start)
            start = loci + 1 + len(i) # adjust subsequence to search for next conserved motive in based upon position of last found motif
        else: continue
    return len(found_univ_conserved_nts)/len(nt_sequence),found_univ_conserved_nts

## training
def pred_eval(y, y_pred):
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel().tolist()
    return tn, fp, fn, tp, tn/(fp+tn), tp/(tp+fn)

def train_test_LR(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return pred_eval(y_test, y_pred)

## timer
class print_time:
    def __init__(self, desc):
        self.desc = desc

    def __enter__(self):
        print(self.desc)
        self.t = time.time()

    def __exit__(self, type, value, traceback):
        print(f'{self.desc} took {time.time()-self.t:.02f}s')

## mutate
def mutate_dna(rna_string, num_mutations):
    rna_list = list(rna_string)
    rna_length = len(rna_list)
    if num_mutations > rna_length:
        raise ValueError("Number of mutations cannot exceed the length of the RNA string.")

    mutation_indices = random.sample(range(rna_length), num_mutations)

    for index in mutation_indices:
        current_base = rna_list[index]
        valid_bases = ['A', 'U', 'C', 'G']
        if current_base not in valid_bases:
            continue
        else:
            valid_bases.remove(current_base) # Ensure the new base is different.
            new_base = random.choice(valid_bases)
            rna_list[index] = new_base

    return "".join(rna_list)  # Convert back to a string

## load
def load_data(filename):
    r = 0.1
    gt = []
    mut = []
    random.seed(42)
    for i in SeqIO.parse(filename,'fasta'):
        s = str(i.seq).upper()
        m = mutate_dna(s, int(r*len(s)))
        gt.append(s)
        mut.append(m)
    return gt, mut

def run_train(gt, mut, func=seq_composition):
    X_gt = []
    X_mut = []

    with print_time('running'):
        for i in range(len(gt)):
            s = gt[i]
            m = mut[i]
            X_gt.append(func(s))
            X_mut.append(func(m))
    
    y_gt = [1] * len(X_gt)
    y_mut = [0] * len(X_mut)
    y = np.array(y_gt + y_mut)
    
    if type(X_gt[0]) is list or type(X_gt[0]) is tuple:
        outs = []
        for i in range(len(X_gt[0])):
            X_gt_i = [x[i] for x in X_gt]
            X_mut_i = [x[i] for x in X_mut]
            X = np.array(X_gt_i + X_mut_i).reshape(-1, 1)
            outs.append(train_test_LR(X, y))
        return outs            
    else:
        X = np.array(X_gt + X_mut).reshape(-1, 1)
        return train_test_LR(X, y)


In [20]:
f = 'SILVA_138.2_SSURef_NR99_tax_silva_filtered.fasta'

In [ ]:
# load
gt, mut = load_data(f)

In [17]:
run_train(gt, mut, seq_composition)

running
running took 39.99s


[(0, 56535, 0, 56423, 0.0, 1.0),
 (29790, 26745, 24566, 31857, 0.5269302202175643, 0.5646101767009908),
 (35670, 20865, 20741, 35682, 0.6309365879543646, 0.6324016801658898),
 (33750, 22785, 22551, 33872, 0.5969753250198991, 0.6003225634936108),
 (30117, 26418, 26649, 29774, 0.5327142478110904, 0.5276926076245503),
 (31159, 25376, 23960, 32463, 0.5511453082161493, 0.5753504776420963),
 (34916, 21619, 22513, 33910, 0.6175997169894756, 0.6009960477110399),
 (37287, 19248, 17339, 39084, 0.6595383390819846, 0.6926962408946706),
 (32598, 23937, 28582, 27841, 0.57659856725922, 0.49343352888006664),
 (36351, 20184, 22303, 34120, 0.6429822234014327, 0.6047179341757793),
 (31752, 24783, 25522, 30901, 0.5616343857787212, 0.5476667316519859)]

In [46]:
def find_homopolymer_and_bias(s):
    hps = find_homopolymer(s)
    return hps + [hps[2] / sum(hps)]

run_train(gt, mut, find_homopolymer_and_bias)

running
running took 175.63s


[(34507, 22028, 23277, 33146, 0.6103652604581233, 0.5874554702869397),
 (32940, 23595, 26946, 29477, 0.5826479172194216, 0.5224287967672757),
 (31712, 24823, 21713, 34710, 0.5609268594675865, 0.6151746628148096),
 (30808, 25727, 26709, 29714, 0.5449367648359423, 0.5266292114917676),
 (35689, 20846, 19872, 36551, 0.6312726629521536, 0.6478032008223596)]

In [ ]:
# this handles all 5 k values in one run
run_train(gt, mut, kmer_parity)

running
running took 7122.57s


[(39403, 17132, 16249, 40174, 0.69696648094101, 0.7120146039735569),
 (38853, 17682, 16175, 40248, 0.687237994162908, 0.7133261258706556),
 (37988, 18547, 19502, 36921, 0.6719377376846202, 0.6543608103078532),
 (34678, 21857, 20365, 36058, 0.6133899354382241, 0.6390656292646616),
 (29732, 26803, 19702, 36721, 0.525904307066419, 0.6508161565319107),
 (40163, 16372, 24606, 31817, 0.7104094808525692, 0.5639012459458023),
 (39968, 16567, 24081, 32342, 0.7069602900857875, 0.5732059621076512),
 (38715, 17820, 26199, 30224, 0.6847970283894932, 0.5356680786204208),
 (34130, 22405, 32463, 23960, 0.6036968249756788, 0.4246495223579037),
 (0, 56535, 0, 56423, 0.0, 1.0)]

In [38]:
run_train(gt, mut, find_universal_primers)

running
running took 85.99s


(55685, 850, 6970, 49453, 0.9849650658883877, 0.8764688159084062)

In [44]:
kmers_of_interest = find_biomarker_kmers(f, klen = 11, min_freq_SILVA = 0.5)

282395it [04:09, 1133.88it/s]


In [67]:
run_train(gt, mut, lambda x: proportion_kmers_present(x, kmers_of_interest))

running
running took 637.06s


(56059, 476, 4742, 51681, 0.9915804368974971, 0.9159562589724048)

In [63]:
kmers = load_gapped_kmers()
kmer_regexes = make_kmer_regexes([kmers[i] for i in range(len(kmers))])

In [66]:
run_train(gt, mut, lambda x: sum([kr.search(x) != None for kr in kmer_regexes]))

running
running took 559.59s


(53436, 3099, 3688, 52735, 0.9451843990448395, 0.9346365843716214)

In [72]:
run_train(gt[:100], mut[:100], predict_mfe_helices)

running
running took 765.51s


[(9, 10, 12, 9, 0.47368421052631576, 0.42857142857142855),
 (10, 9, 10, 11, 0.5263157894736842, 0.5238095238095238)]